# eeg.py 

In [4]:
import airsim
airsimClient = airsim.CarClient()

In [5]:
# -*- coding: utf8 -*-
#
# CyKIT v2 - 2018.Jan.29
# ========================
# Emokit Written by Cody Brocious
# Emokit Written by Kyle Machulis
# CyKIT  Written by Warren
# Contributions  by Severin Lemaignan
# Contributions  by Sharif Olorin
# Contributions  by Bill Schumacher
# Contributions  by CaptainSmiley
#

import time
import os
import sys
import platform
import socket
import struct
import operator
import math
from queue import Queue
import threading

# Import local package: (pywinusb 0.2.9)
sys.path.insert(0, './cyUSB')
import cyUSB as hid
#import pywinusb.hid as hid

# Import local package: (pycrypto 2.6.1)
from Cryptodome.Cipher import AES
from Cryptodome import Random

DEVICE_POLL_INTERVAL = 0.001  # in seconds

tasks = Queue()

class MyIO():
    
    def __init__(self):
        self.integerType = False
        self.noheader = False
        self.ovDelay = 100
        self.ovSamples = 4
        self.openvibe = False
        self.generic = False
        self.format = 0;
        self.update_epoc = None
        self.newMask = None
        self.status = False
        self.setMask = []
        self.setMask = [None]*14
        self.recording = False
        self.recordInc = 1
        self.recordFile = "EEG_recording_"
        self.Delimiter = ", "
        self.samplingRate = 128
        self.channels = 40
        self.f = None
        
    def onData(self, uid, text):
        ioCommand = text.split(":::")
        if ioCommand[0] == "CyKITv2":
            if ioCommand[1] == "setModel":
                self.newModel = int(ioCommand[2])
                print("model=" + ioCommand[2])
            if ioCommand[1] == "changeFormat":
                self.format = int(ioCommand[2])
                if self.format == 1:
                    print("Format Change (Format-1): Javascript handling float conversion.\r\n")
                else:
                    print("Format Change (Format-0): Python handling float conversion.\r\n")
            if ioCommand[1] == "InfoRequest":
                self.server.sendData("CyKITv2:::Info:::Device:::" + str(self.infoDevice))
                self.server.sendData("CyKITv2:::Info:::Serial:::" + str(self.infoSerial))
            if ioCommand[1] == "UpdateSettings":
                self.update_epoc = int(ioCommand[2])
                
            if ioCommand[1] == "RecordStart":
                if self.recording == True:
                    self.recording = False
                    try:
                        self.f.flush()
                        os.fsync(self.f.fileno())
                        self.f.seek(0, os.SEEK_END)
                        f_size = self.f.tell()
                        self.f.truncate((f_size -2))
                        self.f.close()
                    except:
                        pass
                    
                    print("[Record Stopped] -- Press 'Record' to Record a new file.")
                    return
                    
                print("[Start] Recording to File: " + ioCommand[2])
                self.recordFile = str(ioCommand[2])
                
                pathFinder = './EEG-Logs/' + self.recordFile + '.csv'
                try:
                    while os.path.exists(pathFinder):
                        self.recordInc += 1
                        self.recordFile = self.recordFile + "-" + str(self.recordInc)
                        pathFinder = './EEG-Logs/' + self.recordFile + '.csv'
                        print("[Record: File exists. Changing to: " + self.recordFile + ".csv ]")
                except:
                    pass
                try:
                    self.f = file('./EEG-Logs/' + self.recordFile + '.csv', 'a')
                    self.f = open('./EEG-Logs/' + self.recordFile + '.csv', 'a')
                    
                    csvHeader = ""
                    csvHeader += "title: " + self.recordFile + ", "
                    csvHeader += "recorded: " + str(time.strftime("%d.%m.%y %H.%M.%S, "))
                    csvHeader += "timestamp started:2017-11-21T16:17:43.558438-08:00            , "
                    csvHeader += "sampling:" + str(self.samplingRate) + ", "
                    csvHeader += "subject:, "
                    csvHeader += "labels:COUNTER INTERPOLATED "
                    if self.KeyModel == 3 or self.KeyModel == 4:
                        # Insight
                        csvHeader += "AF3 T7 Pz T8 AF4 RAW_CQ GYROX GYROY MARKER SYNC TIME_STAMP_s TIME_STAMP_ms CQ_AF3 CQ_T7 CQ_Pz CQ_T8 CQ_AF4, "
                    else:
                        # Epoc/Epoc+
                        #csvHeader += "AF3 F7 F3 FC5 T7 P7 O1 O2 P8 T8 FC6 F4 F8 AF4 "
                        csvHeader += "F3 FC5 AF3 F7 T7 P7 O1 O2 P8 T8 F8 AF4 FC6 F4 "
                        csvHeader += "RAW_CQ GYROX GYROY MARKER MARKER_HARDWARE SYNC TIME_STAMP_s TIME_STAMP_ms "
                        csvHeader += "CQ_AF3 CQ_F7 CQ_F3 CQ_FC5 CQ_T7 CQ_P7 CQ_O1 CQ_O2 CQ_P8 CQ_T8 CQ_FC6 CQ_F4 CQ_F8 CQ_AF4 CQ_CMS CQ_DRL, "
                    csvHeader += ", "
                    csvHeader += "chan:" + str(self.channels) + ", "
                    csvHeader += "samples:5000, "
                    csvHeader += "units:emotiv"
                    print(csvHeader, file=self.f)
                    os.fsync(self.f.fileno())
                    self.recording = True
                    
                except Exception as msg:
                    print("Error: " + str(msg))
                    
                    pass
                
            if ioCommand[1] == "RecordStop":
                print("[Stop] Recording ") 
                try:
                    self.f.flush()
                    os.fsync(self.f.fileno())
                    
                    self.f.seek(0, os.SEEK_END)
                    f_size = self.f.tell()
                    #print "xxx:" + str(self.f.read(2))
                    self.f.truncate((f_size -2))
                    
                    
                except Exception as msg:
                    print("Error: " + str(msg))
                    pass
                self.recording = False
            if ioCommand[1] == "setMask":
                try:
                    maskSelect = int(ioCommand[2])
                    self.newMask = maskSelect
                    self.setMask[maskSelect] = list(map(int, str(ioCommand[3]).split(".")))
                except Exception as msg:
                    print("Error: " + str(msg))
            #if ioCommand[1] == "ChangeSettings":
        #self.server.sendData("Incoming Data: %s"%(text,))
        return
    
    def onConnect(self, uid):
        self.status = True
        self.newMask = None
        if self.openvibe == True:
            return
        if self.noheader == True:
            return
        self.server.sendData("CyKITv2:::Connected")
        return
    
    def setOVSamples (self, samples):
        self.ovSamples = int(samples)
        print("OpenVibe Samples: " + str(samples))
        return
    
    def getOVSamples(self):
        return self.ovSamples
        
    def setInteger(self, state):
        self.integerType = state
        if state == True: 
            print("Data Type: Integer")
        else:
            print("Data Type: Float")
        return
    
    def getInteger(self):
        return self.integerType
        
    def setHeader(self, state):
        self.noheader = state
        if state == True:
            print("Header: Disabled")
        else:
            print("Header: Enabled")
        return
    
    def getHeader(self):
        return self.noheader
        
    def setGeneric(self, state):
        self.generic = state
        if state == True:
            print("Generic: Enabled")
        else:
            print("Generic: Disabled")
        return
    
    def getOVDelay(self):
        return int(self.ovDelay)
    
    def setOVDelay(self, delay):
        self.ovDelay = (int(delay) * 100)
        print("OpenVibe Delay: " + str(self.ovDelay))
        return

    def setOpenvibe(self, state):
        self.openvibe = state
        if state == True:
            print("OpenVibe: Enabled")
        if state == False:
            print("OpenVibe: Disabled")
        return        
        
    def getOpenvibe(self):
        return self.openvibe
    
    
    def onGeneric(self, uid):
        self.status = True
        self.generic = True
        if self.openvibe == True:
            return
        if self.noheader == True:
            return
        self.server.sendData("CyKITv2:::Connected")
        return
    
    def sendOVint(self, data):
        self.server.sendOVint(data)
        return
    
    def sendOVfloat(self, data):
        self.server.sendOVfloat(data)
        return
    
    def sendData(self, uid, data):
        if self.openvibe == True:
            return
        self.server.sendData(data)
        return      
        
    def status(self):
        return self.status
    
    def onClose(self, uid):
        self.running = False
        return
    
    def modelChange(self):
        if 'newModel' not in globals():
            return 0
        aModel = self.newModel
        self.newModel = 0
        return self.aModel
     
    def update_epoc_settings(self, change):
        if change == 0:
            return self.update_epoc
        else:
            self.update_epoc = None
            return
    
    def startRecord(self, recordPacket):
        try:
            print(recordPacket, file=self.f)
            self.f.flush()
            os.fsync(self.f.fileno())
        except:
            pass

    def stopRecord(self):
        try:              
            if self.f == None:
                return
            self.f.flush()
            os.fsync(self.f.fileno())
            self.f.seek(0, os.SEEK_END)
            f_size = self.f.tell()
            #print "xxx:" + str(self.f.read(2))
            self.f.truncate((f_size -2))
            self.f.close()            # Remove last line.
            #
            #with open('./EEG-Logs/' + self.recordFile + '.csv', 'r+') as f:
            #    f.seek(0, os.SEEK_END) 
            #    while f.tell() and f.read(2) != '\r\n':
            #        f.seek(-4, os.SEEK_CUR)
            #    f.truncate()
            
        except Exception as msg:
            print("Error: " + str(msg))
                    
            pass
            
    def formatChange(self, newFormat):
        self.format = newFormat
        return
        
    def formatStatus(self):
        return self.format
        
    def isRecording(self):
        return self.recording
    
    def setSampling(self, rate):
        self.samplingRate = int(rate)
        return
    
    def getSampling(self):
        return self.samplingRate
        
    def setChannels(self, total):
        self.channels = int(total)
        return
    
    def getChannels(self):
        return self.channels
    
    def setKeyModel(self, key):
        self.KeyModel = key
        return
    
    def getKeyModel(self):
        return self.KeyModel

    def setDelimiter(self, string):
        self.Delimiter = str(string)
        return
    
    def isGeneric(self):
        return self.generic
        
    def getDelimiter(self):
        return str(self.Delimiter)
    
    def maskChange(self):
        return self.newMask
    
    def getMask(self, select):
        self.newMask = None
        return self.setMask[int(select)]
    
    def setReport(self, report):
        self.report = report
        self.epoc_plus_usb = True
    
    def setInfo(self, info, infoData):
        if info == "Device":
            self.infoDevice = str(infoData)
        if info == "Serial":
            self.infoSerial = str(infoData)
        return
        
    def setServer(self, server):
        self.server = server
        return
        
class EEG(object):
    
    def __init__(self, model, io, config):
        global running
        global myIOinstance
        config = config.lower()
        self.time_delay = .001
        self.KeyModel = model
        self.eeg_devices = []
        self.running = True
        self.counter = "0"
        self.serial_number = ""
        self.lock = threading.Lock()
        self.hid = None
        self.myIOinstance = io
        self.myKey = self.Setup(model, config)
        self.recordInc = 1
        self.thread = threading.Thread(name='eegThread', target=self.run, kwargs={'key': self.myKey, 'myio': self.myIOinstance })
        self.thread.setDaemon = False
        self.stop_thread = False
        self.samplingRate = 128
        self.epoc_plus_usb = False
        self.report = None
        self.Delimiter = ", "
        self.channels = 40
        self.blankCSV = False
        self.generic = False
        self.openvibe = False
        self.integerType = False
        
        self.mask = {}
        self.mask[0] = [10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7]
        self.mask[1] = [28, 29, 30, 31, 16, 17, 18, 19, 20, 21, 22, 23, 8, 9]
        self.mask[2] = [46, 47, 32, 33, 34, 35, 36, 37, 38, 39, 24, 25, 26, 27]
        self.mask[3] = [48, 49, 50, 51, 52, 53, 54, 55, 40, 41, 42, 43, 44, 45]
        self.mask[4] = [66, 67, 68, 69, 70, 71, 56, 57, 58, 59, 60, 61, 62, 63]
        self.mask[5] = [84, 85, 86, 87, 72, 73, 74, 75, 76, 77, 78, 79, 64, 65]
        self.mask[6] = [102, 103, 88, 89, 90, 91, 92, 93, 94, 95, 80, 81, 82, 83]
        self.mask[7] = [140, 141, 142, 143, 128, 129, 130, 131, 132, 133, 134, 135, 120, 121]
        self.mask[8] = [158, 159, 144, 145, 146, 147, 148, 149, 150, 151, 136, 137, 138, 139]
        self.mask[9] = [160, 161, 162, 163, 164, 165, 166, 167, 152, 153, 154, 155, 156, 157]
        self.mask[10] = [178, 179, 180, 181, 182, 183, 168, 169, 170, 171, 172, 173, 174, 175]
        self.mask[11] = [196, 197, 198, 199, 184, 185, 186, 187, 188, 189, 190, 191, 176, 177]
        self.mask[12] = [214, 215, 200, 201, 202, 203, 204, 205, 206, 207, 192, 193, 194, 195]
        self.mask[13] = [216, 217, 218, 219, 220, 221, 222, 223, 208, 209, 210, 211, 212, 213]
        
        if "blankdata" in config:     self.blank_data = True
        else:                         self.blank_data = False
        
        if "blankcsv" in config:      self.blankCSV = True
        else:                         self.blankCSV = False
        
        if "nocounter" in config:     self.no_counter = True
        else:                         self.no_counter = False
                    
        if "nobattery" in config:     self.nobattery = True
        else:                         self.nobattery = False
                            
        if "baseline" in config:      self.baseline = True
        else:                         self.baseline = False
        
        if "noheader" in config:      self.noheader = True
        else:                         self.noheader = False
                        
        if "integer" in config:       self.integerType = True
        else:                         self.integerType = False
                
        if "outputdata" in config:    self.outputData = True
        else:                         self.outputData = False
        
        if "generic" in config:       self.generic = True
        else:                         self.generic = False
        
        if "openvibe" in config:      self.openvibe = True
        else:                         self.openvibe = False
        
        if "outputencrypt" in config: self.outputEncrypt = True
        else:                         self.outputEncrypt = False
        
        if "ovdelay" in config:      
            myDelay = str(config).split("ovdelay:")
            self.ovDelay = myDelay[1][:3]
        else:                        
            self.ovDelay = 100
            
        if "ovsamples" in config:      
            mySamples = str(config).split("ovsamples:")
            self.ovSamples = int(mySamples[1][:3])
            if self.ovSamples > 512:
                self.ovSamples = 512
        else:                        
            self.ovSamples = 4
        
        if "format" in config:     
            myFormat = str(config).split("format-")
            self.format = int(myFormat[1][:1])
        else:
            self.format = 0
            
        print("Format: " + str(self.format))
        self.myIOinstance.setOVSamples(self.ovSamples)
        self.myIOinstance.setOVDelay(self.ovDelay)
        self.myIOinstance.setInteger(self.integerType)
        self.myIOinstance.formatChange(self.format)
        self.myIOinstance.setHeader(self.noheader)
        self.myIOinstance.setGeneric(self.generic)
        self.myIOinstance.setOpenvibe(self.openvibe)
        
    def start(self):
        for t in threading.enumerate():
            if 'eegThread' == t.getName():
                return
        self.running = True
        self.thread.start()
        return self.myIOinstance

    
    def Setup(self, model, config):
        # 'EPOC BCI', 'Brain Waves', 'Brain Computer Interface USB Receiver/Dongle', 'Receiver Dongle L01'
        deviceList = ['EPOC+','EEG Signals', '00000000000', 'Emotiv RAW DATA']
        devicesUsed = 0
        
        threadMax = 0
        for t in threading.enumerate():
            if t.getName()[:6] == "Thread": 
                threadMax += 1
                
        for device in hid.find_all_hid_devices():
            if "info" in config:
                print("Product name " + device.product_name)
                print("device path " + device.device_path)
                print("instance id " + device.instance_id)
                print("_" * 80 + "\r\n")
            useDevice = ""
            for i, findDevice in enumerate(deviceList):
                
                if device.product_name == deviceList[i]:
                    print("\r\n>>> Found EEG Device >>> " +  findDevice + "\r\n")
                    if "confirm" in config:
                        useDevice = input("Use this device? [Y]es? ")
                    else:
                        useDevice = "Y"
                    if useDevice.upper() == "Y":
                        devicesUsed += 1
                        self.hid = device
                        if threadMax < 2:
                            self.hid.open()
                        self.serial_number = device.serial_number
                        if threadMax < 2:
                            device.set_raw_data_handler(self.dataHandler)
                        print("> Using Device: " + device.product_name + "\r\n")
                        print("  Serial Number: " + device.serial_number + "\r\n\r\n")
                        if device.product_name == 'EPOC+':
                            deviceList[1] = 'empty'
                            #self.myIOinstance.setReport("Device", device.find_output_reports())
                            
                        #print str(self.report)
        if devicesUsed == 0 or i == 0:
            print("\r\n> No Device Selected. Exiting . . .")
            os._exit(0)
        
        self.myIOinstance.setInfo("Device", device.product_name)
        self.myIOinstance.setInfo("Serial", device.serial_number)
            
        sn = self.serial_number
        
        k = ['\0'] * 16
        
        
        # --- Model 1 > [Epoc::Research]
        if model == 1:
            k = [sn[-1],'\0',sn[-2],'H',sn[-1],'\0',sn[-2],'T',sn[-3],'\x10',sn[-4],'B',sn[-3],'\0',sn[-4],'P']
            self.samplingRate = 128
            self.channels = 40
            
        # --- Model 2 > [Epoc::Standard]
        if model == 2:   
            k = [sn[-1],'\0',sn[-2],'T',sn[-3],'\x10',sn[-4],'B',sn[-1],'\0',sn[-2],'H',sn[-3],'\0',sn[-4],'P']
            self.samplingRate = 128
            self.channels = 40
            
        # --- Model 3 >  [Insight::Research]
        if model == 3:
            k = [sn[-2],'\0',sn[-1],'D',sn[-2],'\0',sn[-1],'\x0C',sn[-4],'\0',sn[-3],'\x15',sn[-4],'\0',sn[-3],'X']
            self.samplingRate = 128
            self.channels = 20
            
        # --- Model 4 > [Insight::Standard]
        if model == 4: 
            k = [sn[-1],'\0',sn[-2],'\x15',sn[-3],'\0',sn[-4],'\x0C',sn[-3],'\0',sn[-2],'D',sn[-1],'\0',sn[-2],'X']
            self.samplingRate = 128
            self.channels = 20
            
        # --- Model 5 > [Epoc+::Research]
        if model == 5:
            k = [sn[-2],sn[-1],sn[-2],sn[-1],sn[-3],sn[-4],sn[-3],sn[-4],sn[-4],sn[-3],sn[-4],sn[-3],sn[-1],sn[-2],sn[-1],sn[-2]]
            self.samplingRate = 256
            self.channels = 40
            
        # --- Model 6 >  [Epoc+::Standard]
        if model == 6:
            k = [sn[-1],sn[-2],sn[-2],sn[-3],sn[-3],sn[-3],sn[-2],sn[-4],sn[-1],sn[-4],sn[-2],sn[-2],sn[-4],sn[-4],sn[-2],sn[-1]]
            self.samplingRate = 256
            self.channels = 40
            
        self.myIOinstance.setSampling(self.samplingRate)
        self.myIOinstance.setChannels(self.channels)
        self.myIOinstance.setKeyModel(model)
        
        key = ''.join(k)
        print("key = " + str(key))
        return str(key)
            

    def dataHandler(self, data):
        try:
            if self.blank_data == True:
                if data != "": return
                data = [0, 11, 45, 226, 13, 209, 11, 156, 77, 16, 118, 83, 208, 255, 75, 10, 40, 241, 206, 231, 146, 226, 59, 124, 165, 69, 24, 248, 163, 55, 25, 133, 167]
        
            if self.outputEncrypt == True:
                print(str(data))
        except:
            pass
        tasks.put(''.join(map(chr, data[1:])))
        return True
  
    def convertEPOC(self, data, bits):
        level = 0
        for i in range(13, -1, -1):
            level <<= 1
            b, o = (bits[i] / 8) + 1, bits[i] % 8
            level |= (ord(data[b]) >> o) & 1
        return level
    
    def convertEPOC_PLUS(self, value_1, value_2):
        
        edk_value = "%.8f" % (((int(value_1) * .128205128205129) + 4201.02564096001) + ((int(value_2) -128) * 32.82051289))
        if self.integerType == True:
            return str(int(float(edk_value)))
        #edk_value = "%.6f" % (((int(value_2) * .128205148) + 4201.02564096001) + ((int(value_1) -128) * 32.82051286))
        return edk_value
         
    def run(self, key, myio):
        
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(key, AES.MODE_ECB, iv)
        
        self.lock.acquire()
        thread_name = ""
        print("\r\nActive Threads = {")
        for t in threading.enumerate():
            try:
                thread_name = str(t).split("(")
            except:
                continue
            
            print("   " + str(thread_name[0]) + " ::: " + str(t.getName()) + ">")
        print("} \r\n")
        self.lock.release()

        # Send information to CyInterface.
        
        self.Delimiter = str(self.myIOinstance.getDelimiter())
        

        if self.myIOinstance.getHeader() == False:
            if self.myIOinstance.status == True:
                myio.sendData(1, "CyKITv2:::Info:::Device:::" + str(self.hid.product_name))
                myio.sendData(1, "CyKITv2:::Info:::Serial:::" + str(self.hid.serial_number))
                myio.sendData(1, "CyKITv2:::Info:::KeyModel:::" + str(self.KeyModel))
                myio.sendData(1, "CyKITv2:::Info:::Delimiter:::" + str(self.Delimiter))
            
        self.generic = self.myIOinstance.isGeneric()
        
        while self.running:
            if self.myIOinstance.status != True:
                return
            
            if self.myIOinstance.update_epoc_settings(0) != None:
                try:
                    EPOC_ChangeMode = self.myIOinstance.update_epoc_settings(0)
                    self.myIOinstance.update_epoc_settings(1);
                    print(str(EPOC_ChangeMode))
                    ep_mode = [0x0] * 32
                    ep_mode[1:4] = [0x55,0xAA,0x20,0x12] 
                    ep_select = [0x00,0x82,0x86,0x8A,0x8E,0xE2,0xE6,0xEA,0xEE]
                    ep_mode[5] = ep_select[EPOC_ChangeMode]
                    print(str(ep_mode))
                    print(str(len(ep_mode)))
                    #0 EPOC                                  0x00 (d.000)
                    #1 EPOC+ 128hz 16bit - MEMS off          0x82 (d.130)
                    #2 EPOC+ 128hz 16bit - MEMS 32hz 16bit   0x86 (d.134)
                    #3 EPOC+ 128hz 16bit - MEMS 64hz 16bit   0x8A (d.138)
                    #4 EPOC+ 128hz 16bit - MEMS 128hz 16bit  0x8E (d.142)
                    #5 EPOC+ 256hz 16bit - MEMS off          0xE2 (d.226)
                    #6 EPOC+ 256hz 16bit - MEMS 32hz 16bit   0xE6 (d.230)
                    #7 EPOC+ 256hz 16bit - MEMS 64hz 16bit   0xEA (d.234)
                    #8 EPOC+ 256hz 16bit - MEMS 128hz 16bit  0xEE (d.238)
                    
                    report = self.hid.find_output_reports()
                    report[0].set_raw_data(ep_mode)
                    report[0].send()
                    print("sending packet!")
                    
                
                except Exception as exception:
                    print(("Oops!",sys.exc_info()[0],"occured."))
                    print(exception)
                    print(sys.exc_traceback.tb_lineno) 
                
              
            if self.blank_data == True:
                self.dataHandler("")
            
            while not tasks.empty():
                check_mask = self.myIOinstance.maskChange()
                
                self.format = self.myIOinstance.formatStatus();
                
                if check_mask != None:
                    self.mask[check_mask] = self.myIOinstance.getMask(check_mask)
                    print(self.mask[check_mask])
                task = tasks.get()
                
                try:
                    data = cipher.decrypt(task[:16]) + cipher.decrypt(task[16:])
                    
                    counter_data = ""
                    packet_data = ""
                    
                    if self.KeyModel == 6 or self.KeyModel == 5:
                        
                        if self.no_counter == True:
                            counter_data = ""
                        else:
                            counter_data = str(ord(data[0])) + self.Delimiter + str(ord(data[1])) + self.Delimiter
                                
                        # Format 0: Default.  
                        if self.format < 1:
                            for i in range(2,16,2):
                                packet_data = packet_data + str(self.convertEPOC_PLUS(str(ord(data[i])), str(ord(data[i+1])))) + self.Delimiter
                            
                            for i in range(18,len(data),2):
                                packet_data = packet_data + str(self.convertEPOC_PLUS(str(ord(data[i])), str(ord(data[i+1])))) + self.Delimiter
                            
                            packet_data = packet_data[:-len(self.Delimiter)]
                            
                            if self.baseline == True:
                                if baseline_values != None:
                                    baseline_last = baseline_values
                                baseline_values = packet_data.split(self.Delimiter)
                                
                                if baseline_values != None:
                                    baseline_values = list(map(operator.add, baseline_last, baseline_values))
                                    baseline_values = list(map(operator.div, baseline_value, ([2] * len(base_values))))
                                
                                print(str(baseline_values))
                            #if self.quality == True:
                            #baseline_values = map(math.sqrt, baseline_values)
                                
                                
                                print(str(baseline_values))
                            
                            if self.nobattery == False:
                                    packet_data = packet_data + self.Delimiter + str(ord(data[16])) + str(self.Delimiter) + str(ord(data[17])) 
                            
                            if self.myIOinstance.isRecording() == True:
                                record_data = packet_data
                                if self.blankCSV == True:
                                    
                                    emptyCSV = ("0" + self.Delimiter) * int(self.channels - (16 + abs((self.nobattery & 1) *-2)))
                                    
                                    emptyCSV = emptyCSV[:-2]
                                    record_data = packet_data + self.Delimiter + emptyCSV
                                    
                                try:
                                    airsim_data = airsimClient.getAdasPacket()
                                except:
                                    print('airsim v2 needed')
                                    airsim_data = 'airsim v2 needed'
                                
                                self.myIOinstance.startRecord(airsim_data + counter_data + record_data)
                            
                            if self.outputData == True:
                                print(str(airsim_data + counter_data + packet_data))
                        
                    try:
                        myio.sendData(1, counter_data + packet_data)
                    except Exception as msg:
                        if str(msg[0]) == "10035":
                            self.time_delay += .001
                            time.sleep(self.time_delay)
                            continue
                            
                        if msg[0] == 9 or msg[0] == 10053 or msg[0] == 10035 or msg[0] == 10054:
                            print(str(msg))
                            print("\r\nConnection Closing.\r\n")
                            self.running = False
                            tasks.queue.clear()
                            if self.generic == True:
                                myio.onClose(0)
                            else:
                                myio.onClose(1)
                            self.hid.close()
                            self.myIOinstance.stopRecord()
                            continue
                        print("eeg().run() Error: " + str(msg))

                except Exception as exception2:
                    print(str(exception2))


In [6]:
# -*- coding: utf8 -*-
# 
# pywebsocketserver  2018.Jan.29
# ================================
# Written  by suxianbaozi
#
# CyWebSocket.py     2018.Jan.29
# ================================
# Modified by Warren
#
# Python web server for connecting sockets locally with browsers 
# as well as a generic TCP server.
#

import sys
import socket
import time
import select
import threading
import hashlib
import base64
import struct

class socketIO():
    
    def __init__(self, port, uid, ioHandler):
        self.time_delay = .001
        self.openvibe = False
        self.ovDelay = 100
        self.port = port
        self.con = None
        self.isHandleShake = False
        self.uid = uid
        self.io = ioHandler
        self.signKey = "ADS#@!D"
        self.online = True
        self.generic = False
        self.lock = threading.Lock()
        self.thread = threading.Thread(name='ioThread', target=self.run)
        self.thread.setDaemon = False
        self.stop_thread = False
        self.ovData = ""
        self.ovSamples = 4
        
#         if uid == 0:
#             self.generic = True
#             self.isHandleShake = True
#             self.isGeneric = True
        
    def start(self):
        self.socketThreadRunning = True
        for t in threading.enumerate():
            if 'ioThread' == t.getName():
                return
        self.thread.start()

        
    def Handshake(self):
        self.isHandleShake = False
        self.online = True
        self.socketThreadRunning = True
        
    def Connect(self):
        print("* Connecting . . .")
        sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        sock.bind(('',self.port))
        sock.listen(1)
        
        try:
            connection,address = sock.accept()
            self.con = connection
            print("> Connected!")
        except:
            print("> Not Connected -" + sock.error)
        
        return self.con
            

    def run(self):
            
        self.socketThreadRunning = True
        
        if self.io.getOpenvibe() == True:
                self.io.onGeneric(0)
                self.ovDelay = self.io.getOVDelay()
                self.ovSamples = self.io.getOVSamples()
                
        while self.socketThreadRunning == True:
            
            if self.io.getOpenvibe() == True:
                self.openvibeTimer = 0
                while self.openvibeTimer > self.ovDelay:
                    self.openvibeTimer += 1
                return
            
            if self.generic == True:
                try:
                    self.con.setblocking(0)
                    ready = select.select([self.con], [], [], 1)
                    if ready[0]:
                        clientData  = self.con.recv(1024)
                        self.io.onGeneric(0)
                        if self.io.getOpenvibe() == True:
                            self.openvibe = True
                        continue
                        
                    continue
                except socket.error as e:
                    print(str(socket.error))
                    if e[0] == 10035:
                        self.time_delay += .001
                        time.sleep(self.time_delay)
                        continue
                    continue
            
                    
            if not self.isHandleShake: 
                print("trying this!")
                try:
                    self.con.setblocking(0)
                    ready = select.select([self.con], [], [], 1)
                    if ready[0]:
                        clientData  = self.con.recv(1024)
                        
                        #print clientData
                        dataList = clientData.split("\r\n")
                        header = {}
                        for data in dataList:
                            if ": " in data:
                                unit = data.split(": ")
                                header[unit[0]] = unit[1]
                        secKey = header['Sec-WebSocket-Key'];
                        resKey = base64.encodestring(hashlib.new("sha1",secKey+"258EAFA5-E914-47DA-95CA-C5AB0DC85B11").digest());
                        resKey = resKey.replace('\n','')
                        response = '''HTTP/1.1 101 Switching Protocols\r\n'''
                        response += '''Upgrade: websocket\r\n'''
                        response += '''Connection: Upgrade\r\n'''
                        response += '''Sec-WebSocket-Accept: %s\r\n\r\n'''%(resKey,)
                        self.con.send(response)
                        self.isHandleShake = True
                        self.sendData("SETUID")
                        self.io.onConnect(self.uid)
                        continue
                except:
                    continue
                    
            else:
                try:
                    
                    ready = select.select([self.con], [], [], 0)
                    if ready[0]:
                        data_head = self.con.recv(1)
                        
                        if repr(data_head)=='':
                            self.onClose()
                            continue
                        
                        header = struct.unpack("B",data_head)[0]
                        opcode = header & 0b00001111

                        if opcode == 8:
                            print("* Closing Connection.")
                            self.socketThreadRunning = False
                            self.onClose()
                            
                            continue
                        
                        data_length = self.con.recv(1)
                        data_lengths= struct.unpack("B",data_length)
                        data_length = data_lengths[0]& 0b01111111
                        masking = data_lengths[0] >> 7
                        if data_length<=125:
                            payloadLength = data_length
                        elif data_length==126:
                            payloadLength = struct.unpack("H",self.con.recv(2))[0]
                        elif data_length==127:
                            payloadLength = struct.unpack("Q",self.con.recv(8))[0]
                        print("dataLen:%d"%(data_length,))
                        if masking==1:
                            maskingKey = self.con.recv(4)
                            self.maskingKey = maskingKey
                        data = self.con.recv(payloadLength)
                        if masking==1:
                            i = 0
                            true_data = ''
                            for d in data:
                                true_data += chr(ord(d) ^ ord(maskingKey[i%4]))
                                i += 1
                            self.onData(true_data)
                        else:
                            self.onData(data)
                    
                        
                except Exception as msg:
                    if msg[0] == 10035:
                        self.time_delay += .001
                        time.sleep(self.time_delay)
                        continue
                    if msg[0] == 9 or msg[0] == 10053 or msg[0] == 10054:
                        self.socketThreadRunning = False
                    
                    print("CyWebSocket().socketIO() Error: " + str(msg))
                    self.socketThreadRunning = False
                    self.onClose()
                    
                    return
            
            
    def onData(self,text) :
        print(text)
        try:
            uid,sign,value = text.split("<split>")
            uid = int(uid)
            print(str(text))
        except:
            print("Error")
            self.con.close()
        hashStr = hashlib.new("md5",str(uid)+self.signKey).hexdigest()
        if hashStr!=sign:
            print("Hash Invalid")
            self.con.close()
            return
        return self.io.onData(uid,value)

    def onClose(self):
        self.con.close()
        self.online = False
        self.io.onClose(self.uid)

    def packData(self,text):
        sign = hashlib.new("md5",str(self.uid)+self.signKey).hexdigest()
        data = '%s<split>%s<split>%s'%(self.uid,sign,text)
        return data
    
    def sendOVint(self, text):
        if len(self.ovData) >= (self.ovSamples *28):
            self.con.sendall(self.ovData)
            self.ovData = ""
            
        ov_split = str(text).split(", ")
        #ov_floats = map((lambda x: float("%.11f" % float(x))), ov_split)
        ov_ints = [int(float(x)) for x in ov_split]
        self.ovData = struct.pack('>' + ('h' * len(ov_ints)), *ov_ints)

        self.con.sendall(self.ovData)
    
    def sendOVfloat(self, text):
        if len(self.ovData) >= (self.ovSamples *56):
            self.con.sendall(self.ovData)
            self.ovData = ""
            
        ov_split = str(text).split(", ")
        #ov_floats = map((lambda x: float("%.11f" % float(x))), ov_split)
        ov_floats = [float(x) for x in ov_split]
        self.ovData += struct.pack('>' + ('f' * len(ov_floats)), *ov_floats)
        
    
    def sendData(self, text):
        if self.uid == 0:
            self.con.send(text + "\r\n")
        else: 
            text = self.packData(text)
            self.con.send(struct.pack("!B",0x81))
            length = len(text)

            if length<=125:
                self.con.send(struct.pack("!B",length))

            elif length<=65536:
                self.con.send(struct.pack("!B",126))
                self.con.send(struct.pack("!H",length))
            else:
                self.con.send(struct.pack("!B",127))
                self.con.send(struct.pack("!Q",length))

            self.con.send(struct.pack("!%ds"%(length,),text))
        
        


In [ ]:
# -*- coding: utf8 -*-
#
# CyKIT v2 - 2018.Jan.29
# ========================
# Written by Warren
#

import sys
import socket
import select
import struct
import eeg
import CyWebSocket
import threading
import time

# arg_count = len(sys.argv)
# if arg_count == 1 or arg_count > 5 or sys.argv[1] == "help" or sys.argv[1] == "--help" or sys.argv[1] == "/?":
#     print("\r\n")
#     print(" (Version: CyKITv2:2018.Jan.29) -- Python 2.7.6 on Win32 \r\n")
#     print("\r\n Usage:  Python.exe CyKITv2.py <IP> <Port> <Model#(1-6)> [config] \r\n")
#     print(" " + "_" * 85 + "\r\n")
#     print(" <IP> <PORT> for CyKIT to listen on. \r\n") 
#     print(" " + "_" * 85 + "\r\n")
#     print(" <Model#> Choose the decryption type. \r\n")
#     print("          1 - Epoc (Research)\r\n")
#     print("          2 - Epoc (Standard)\r\n")
#     print("          3 - Insight (Research)\r\n")
#     print("          4 - Insight (Standard)\r\n")
#     print("          5 - Epoc+ (Research)\r\n")
#     print("          6 - Epoc+ (Standard)\r\n\r\n")
#     print(" " + "_" * 85 + "\r\n")
#     print(" [config] is optional. \r\n")
#     print("  'info'          Prints additional information into console.\r\n")
#     print("  'confirm'       Requests you to confirm a device everytime device is initialized.\r\n")
#     print("  'nocounter'     Removes all counters from 'all' outputs.\r\n")
#     print("  'noheader'      Removes CyKITv2::: header information. (Required for openvibe) \r\n")
#     print("  'format-0'      (Default) Outputs 14 data channels in float format. ('4201.02564096') \r\n")
#     print("  'format-1'      Outputs the raw data (to be converted by Javascript or other). \r\n")
#     print("  'outputdata'    Prints the (formatted) data being sent, to the console window.\r\n")
#     print("  'outputencrypt' Prints the (encrypted) rjindael data to the console window.\r\n\r\n")
#     print("  'blankdata'     Injects a single line of encrypted data into the stream that is \r\n")
#     print("                   consistent with a blank EEG signal. Counter will report 0. \r\n\r\n")
#     print("  'blancsv'       Adds blank channels for each CSV line, to be used with logging.\r\n\r\n")
#     print("  'generic'       Connects to any generic program via TCP. (Can be used with other flags.)\r\n\r\n")
#     print("  'openvibe'      Connects to the generic OpenViBE Acquisition Server.\r\n\r\n")
#     print("                  must use generic+nocounter+noheader+nobattery Other flags are optional.\r\n")
#     print("  'ovdelay'       Stream sending delay. (999 maximum) Works as a multiplier, in the format: ovdelay:001 \r\n\r\n")
#     print("  'ovsamples'     Changes openvibe sample rate. Format: ovsamples:001 \r\n\r\n")
#     print("  'integer'       Changes format from float to integer. Works with other flags. Including openvibe. \r\n\r\n")
#     print("   Join these words together, using a + separator. \r\n")
#     print("   (e.g  info+confirm ) \r\n\r\n")
#     print(" " + "_" * 85 + "\r\n")
#     print("  Example Usage: \r\n")
#     print("  Python.exe CyKITv2.py 127.0.0.1 55555 1 info+confirm \r\n\r\n")    
#     print("  Example Usage: \r\n")
#     print("  Python.exe CyKITv2.py 127.0.0.1 55555 6 openvibe+generic+nocounter++noheader+nobattery+ovdelay:100+integer+ovsamples:004 \r\n\r\n")
#     print(" " + "_" * 85 + "\r\n")
#     sys.argv = [sys.argv[0], "127.0.0.1", "55555", "1", ""]
    
    
# if arg_count < 5:
    
#     if arg_count == 2:
#         sys.argv = [sys.argv[0], sys.argv[1], "55555", "1", ""]
#     if arg_count == 3:
#         sys.argv = [sys.argv[0], sys.argv[1], sys.argv[2], "1", ""]
#     if arg_count == 4:
#         sys.argv = [sys.argv[0], sys.argv[1], sys.argv[2], sys.argv[3], ""]
   
    
   
def main(CyINIT):
    
#     parameters = str(sys.argv[4]).lower()
    
    if 'CyINIT' not in locals():
        #global CyINIT
        CyINIT = 2
   
    CyINIT += 1
    HOST = '127.0.0.1'
    PORT = int(55555)
    MODEL = int(6)
    
    # Initialize CyKIT 
    if CyINIT == 2:
        global ioTHREAD
        print("> Listening on " + HOST + " : " + str(PORT))
        print("> Trying Key Model #: " + str(MODEL))
        
        myi = eeg.MyIO()
        
#         if "noheader" in parameters:
#             myi.setHeader(True)
#         if "openvibe" in parameters:
#             myi.setOpenvibe(True)
#         if "generic" in parameters:
#             ioTHREAD = CyWebSocket.socketIO(PORT, 0, myi)
#         else:
        ioTHREAD = CyWebSocket.socketIO(PORT, 1, myi)
        myi.setServer(ioTHREAD)
        check_connection = ioTHREAD.Connect()
        cyIO = ioTHREAD.start()
        
        cyHeadset = eeg.EEG(MODEL, myi, '').start()
        for t in threading.enumerate():
            print(str(t.getName()))
        CyINIT += 1
        if myi.getOpenvibe() == True:
            time.sleep(3)
        
    while CyINIT > 2:
        CyINIT += 1
        
        if CyINIT > 1000:
            modelCheck = myi.modelChange()
            if modelCheck != 0:
                MODEL = modelCheck
            
            CyINIT = 3
            check_threads = 0
            #print "testing"
            
            for t in threading.enumerate():
                if t.getName() == "ioThread" or t.getName() == "eegThread":
                    check_threads += 1
            
            if myi.getOpenvibe() == True:
                if check_threads == 0:
                    ioTHREAD.onClose()
                    print("*** Reseting . . .")
                    CyINIT = 1
                    main(1)
                continue
                
            
            if check_threads == 1:
                ioTHREAD.onClose()
                print("*** Reseting . . .")
                CyINIT = 1
                main(1)
    
try:
    
    main(1)
  
except Exception as e:
    print(e)
    print("Device Time Out or Disconnect . . .    Reconnect to Server.")
    main(1)